In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time


In [3]:
driver_path = 'C:\Program Files (x86)\chromedriver.exe'

service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
url = 'https://www.youtube.com/@UnfoldDataScience'
driver.get(url)
driver.maximize_window()
time.sleep(2)
videos_button = driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div/div[1]')
videos_button.click()

 
for _ in range(25):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)

count = 0
video_data = []

videos = driver.find_elements(By.CLASS_NAME, "style-scope ytd-rich-grid-media")

for video in videos:
    title = video.find_element(By.XPATH, './/*[@id="video-title"]').text
    views = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[1]').text
    upload_date = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[2]').text
    link = video.find_element(By.XPATH, './/*[@id="video-title-link"]').get_attribute('href')

    link_array.append(link)
    video_data.append({
        'Title': title,
        'Views': views,
        'Upload Date': upload_date,
        'Link': link
    })

    count += 1

likes_counts = []
comments_counts = []

for link in link_array:
    driver.execute_script("window.open('', '_blank');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

    time.sleep(2)

    # likes_count = driver.find_element(By.XPATH, './/*[@id="segmented-like-button"]').text
    # comments_count = driver.find_element(By.CSS_SELECTOR, '#count button span').text

    # likes_counts.append(likes_count)
    # comments_counts.append(comments_count)

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

driver.quit()

 
for i, video in enumerate(video_data):
    video['Likes Count'] = likes_counts[i]
    video['Comments Count'] = comments_counts[i]

 
df= pd.DataFrame(video_data)

 
df.to_csv('data.csv', index=False)


In [ ]:
import pandas as pd
 
def load_data_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    return df
 
def average_views_last_30_days(df):
   
    df['Upload Date'] = pd.to_datetime(df['Upload Date'])
    current_date = pd.to_datetime('today')
    thirty_days_ago = current_date - pd.DateOffset(days=30)
    recent_videos = df[df['Upload Date'] >= thirty_days_ago]
    average_views = recent_videos['Views'].mean()
    return average_views

 
def highest_likes_to_views_ratio(df):
    df['Likes'] = df['Likes Count'].str.replace('K', 'e3', regex=True).str.replace('M', 'e6', regex=True).map(pd.eval).astype('int64')
    df['Views'] = df['Views'].str.replace('K', 'e3', regex=True).str.replace('M', 'e6', regex=True).map(pd.eval).astype('int64')
    df['Likes-to-Views Ratio'] = df['Likes'] / df['Views']
    max_ratio_index = df['Likes-to-Views Ratio'].idxmax()
    highest_ratio_video = df.loc[max_ratio_index] 
    return highest_ratio_video

def likes_comments_correlation(df):
    df['Likes Count'] = df['Likes Count'].str.replace('K', 'e3', regex=True).str.replace('M', 'e6', regex=True).map(pd.eval).astype('int64')
    df['Comments Count'] = df['Comments Count'].str.replace('K', 'e3', regex=True).str.replace('M', 'e6', regex=True).map(pd.eval).astype('int64')
    correlation = df['Likes Count'].corr(df['Comments Count'])
    
    return correlation

 
def common_day_of_week(df):
    df['Upload Date'] = pd.to_datetime(df['Upload Date'])
    df['Day of Week'] = df['Upload Date'].dt.day_name()
    common_day = df['Day of Week'].mode().values[0]
    
    return common_day
 
def detect_outliers(df, z_threshold=2.0):
    df['Views'] = df['Views'].str.replace('K', 'e3', regex=True).str.replace('M', 'e6', regex=True).map(pd.eval).astype('int64')
    df['Z-Score'] = (df['Views'] - df['Views'].mean()) / df['Views'].std()
    outliers = df[df['Z-Score'].abs() > z_threshold]
    
    return outliers

 
def main():
    csv_file = 'data.csv'
    data = load_data_from_csv(csv_file)
    avg_views_last_30 = average_views_last_30_days(data)
    print(f'Average Views Count for Videos Uploaded in the Last 30 Days: {avg_views_last_30:.2f}')
    highest_ratio_video = highest_likes_to_views_ratio(data)
    print(f'Video with the Highest Likes-to-Views Ratio:\n{highest_ratio_video}')
    correlation = likes_comments_correlation(data)
    print(f'Correlation between Likes Count and Comments Count: {correlation:.2f}')
    common_upload_day = common_day_of_week(data)
    print(f'Most Common Day of the Week for Video Uploads: {common_upload_day}') 
    outliers = detect_outliers(data, z_threshold=2.0)
    print('Outliers in Views Count:\n', outliers)

if __name__ == '__main__':
    main()
